In [184]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from imutils import paths
import imagesize
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse
import random
import pickle
import cv2
import os

In [156]:
class modelNet:
    @staticmethod
    def LeNet(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        
        model.add(Conv2D(4, (3, 3), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        
        # model.add(Conv2D(16, (5, 5), padding="same"))
        # model.add(Activation("relu"))
        # model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        model.add(Flatten())
        model.add(Dense(5))
        model.add(Activation("relu"))
        
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model
    
    @staticmethod
    def VGG16(classes):
        # initializing the model
        model = Sequential()
        
        
        model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
        model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
        
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        
        model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        
        model.add(Flatten())
        model.add(Dense(units=4096,activation="relu"))
        model.add(Dense(units=4096,activation="relu"))
        
        model.add(Dense(units=classes, activation="softmax"))
    

In [157]:
EPOCHS = 10
INIT_LR = 1e-3
BS = 8
CLASSES = 3

In [158]:
print("[INFO] loading images...")
data = []
labels = []

[INFO] loading images...


In [159]:
imagePaths = sorted(list(paths.list_images("augData/trialDataset")))
random.seed(42)
random.shuffle(imagePaths)

### Visualizing image Size

In [186]:
img_meta = {}
for path in imagePaths: img_meta[path] = imagesize.get(path)

# converting the sizes into graph
img_meta_df = pd.DataFrame.from_dict([img_meta]).T.reset_index().set_axis(['FileName', 'Size'], axis='columns', inplace=False)
img_meta_df[["Width", "Height"]] = pd.DataFrame(img_meta_df["Size"].tolist(), index=img_meta_df.index)
img_meta_df["Aspect Ratio"] = round(img_meta_df["Width"] / img_meta_df["Height"], 2)

print(f'Total Nr of Images in the dataset: {len(img_meta_df)}')
img_meta_df.head()

Total Nr of Images in the dataset: 2522


,FileName,Size,Width,Height,Aspect Ratio
0,augData/trialDataset/double_person/image174 (c...,"(165, 627)",165,627,0.26
1,augData/trialDataset/triple_person/image_0_747...,"(338, 519)",338,519,0.65
2,augData/trialDataset/triple_person/image_0_882...,"(249, 507)",249,507,0.49
3,augData/trialDataset/double_person/image131 (c...,"(233, 436)",233,436,0.53
4,augData/trialDataset/triple_person/image_0_312...,"(249, 507)",249,507,0.49


In [190]:
# Visualize Image Resolutions

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
points = ax.scatter(img_meta_df.Width, img_meta_df.Height, color='blue', alpha=0.5, s=img_meta_df["Aspect Ratio"]*100, picker=True)
ax.set_title("Image Resolution")
ax.set_xlabel("Width", size=14)
ax.set_ylabel("Height", size=14)
plt.savefig("img_plot.png")

In [160]:
for imagePath in imagePaths:    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
    
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    data.append(image)
    

In [161]:
unique, counts = np.unique(np.array(labels), return_counts=True)
dict(zip(unique, counts))

{'double_person': 947, 'single_person': 963, 'triple_person': 612}

In [162]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [163]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [164]:
print("[INFO] compiling model...")
model = modelNet.LeNet(width=28, height=28, depth=3, classes=CLASSES)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [165]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 28, 28, 4)         112       
                                                                 
 activation_21 (Activation)  (None, 28, 28, 4)         0         
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 14, 14, 4)        0         
 2D)                                                             
                                                                 
 flatten_7 (Flatten)         (None, 784)               0         
                                                                 
 dense_14 (Dense)            (None, 5)                 3925      
                                                                 
 activation_22 (Activation)  (None, 5)                 0         
                                                      

In [166]:
print("[INFO] training network...")
H = model.fit(trainX, trainY, batch_size=BS,
    validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)
# save the model to disk
print("[INFO] serializing network...")
model.save("model_1.h5", save_format="h5")

[INFO] training network...
Epoch 1/10
236/236 [==============================] - 6s 18ms/step - loss: 0.9528 - accuracy: 0.4915 - val_loss: 0.8925 - val_accuracy: 0.5547
Epoch 2/10
236/236 [==============================] - 4s 15ms/step - loss: 0.8329 - accuracy: 0.5688 - val_loss: 0.7325 - val_accuracy: 0.6387
Epoch 3/10
236/236 [==============================] - 4s 18ms/step - loss: 0.6785 - accuracy: 0.6925 - val_loss: 0.6411 - val_accuracy: 0.6926
Epoch 4/10
236/236 [==============================] - 4s 17ms/step - loss: 0.6082 - accuracy: 0.7180 - val_loss: 0.6283 - val_accuracy: 0.6799
Epoch 5/10
236/236 [==============================] - 4s 16ms/step - loss: 0.5651 - accuracy: 0.7398 - val_loss: 0.6502 - val_accuracy: 0.6656
Epoch 6/10
236/236 [==============================] - 4s 17ms/step - loss: 0.5471 - accuracy: 0.7483 - val_loss: 0.7174 - val_accuracy: 0.6688
Epoch 7/10
236/236 [==============================] - 4s 18ms/step - loss: 0.5175 - accuracy: 0.7658 - val_loss: 0.

In [167]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy on Double and triple rider")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot_multi_class_4.png")

In [168]:
predictions = model.predict(x=testX, batch_size=32)

print(predictions.shape)

(631, 3)


In [169]:
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1),target_names=lb.classes_))

               precision    recall  f1-score   support

double_person       0.71      0.61      0.66       251
single_person       0.69      0.82      0.75       239
triple_person       0.91      0.85      0.88       141

     accuracy                           0.74       631
    macro avg       0.77      0.76      0.76       631
 weighted avg       0.75      0.74      0.74       631



### Trying out test set with 3 classification

In [170]:
test_imagepaths = sorted(list(paths.list_images("testSet")))
random.seed(42)
random.shuffle(test_imagepaths)

In [171]:
test_labels = []
test_data = []
for imagePath in test_imagepaths:    
    label = imagePath.split(os.path.sep)[-2]
    test_labels.append(label)
    
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    test_data.append(image)

test_labels = np.array(test_labels)
test_data = np.array(test_data, dtype="float") / 255.0


In [172]:
lb = LabelBinarizer()
test_labels = lb.fit_transform(test_labels)

In [173]:
test_predictions = model.predict(x=test_data, batch_size=32)

print(test_predictions.shape)

(221, 3)


In [174]:
test_predictions.argmax(axis=1)

array([0, 0, 2, 2, 0, 0, 1, 0, 2, 1, 2, 0, 1, 1, 1, 1, 0, 2, 2, 0, 2, 2,
       0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 0, 2, 1, 1, 1, 0, 2, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 1, 0, 0, 2, 1, 1, 0, 2, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 2, 2, 1, 2, 0, 0, 0, 1, 1, 2, 1, 0, 1,
       1, 0, 2, 0, 1, 2, 0, 0, 2, 1, 0, 2, 0, 0, 2, 1, 1, 1, 2, 0, 1, 2,
       0, 1, 0, 2, 1, 1, 1, 0, 0, 0, 1, 1, 0, 2, 2, 0, 1, 2, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 2, 0, 2, 2, 0, 1, 1, 2, 1, 1, 0, 0, 2, 2, 0, 1, 2,
       0, 2, 0, 1, 2, 0, 0, 2, 1, 2, 2, 1, 2, 0, 2, 1, 2, 1, 0, 2, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 2, 2, 1, 0, 0, 0, 0, 1, 2, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 2, 2, 0, 1, 2, 0, 2, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 2, 1,
       0])

In [175]:
print(classification_report(test_labels.argmax(axis=1),test_predictions.argmax(axis=1),target_names=lb.classes_))

               precision    recall  f1-score   support

double_person       0.23      0.28      0.26        78
single_person       0.65      0.60      0.62        80
triple_person       0.28      0.24      0.26        63

     accuracy                           0.38       221
    macro avg       0.39      0.37      0.38       221
 weighted avg       0.40      0.38      0.39       221

